# Import dependencies and data

In [ ]:

# Import dependencies
import pandas as pd
import numpy as np
import sklearn
from sklearn import linear_model
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix, precision_recall_curve, auc
import statsmodels.api as sm
import xgboost as xgb
import pickle

# Suppress scientific notation
pd.set_option('display.float_format', lambda x: '%.4f' % x)

# Import data (Source: https://www.kaggle.com/arashnic/imbalanced-data-practice)
data = pd.read_csv('aug_train.csv')

In [ ]:
data.head()

# Inspect data

### Inspect id column

In [ ]:
# Assert that each row contains unique user_id
assert len(data['id'].unique()) == data.shape[0]

### Inspect summary statistics of features

In [ ]:

def inspect_cont_feat(feature):

    # Inspect percent missing, mean, standard dev, min, and max of continuous feature
    feature_stats = {
        'pct_null': np.round(feature.isnull().sum() * 100 / len(feature), 2),
        'mean': np.round(np.mean(feature), 2),
        'sd': np.round(np.std(feature), 2),
        'min': np.round(np.min(feature), 2),
        'max': np.round(np.max(feature), 2)}
    
    # Print feature summary statistics
    print('-'*40)
    print('Feature: {}'.format(feature.name))
    print('Percent null: {}, mean: {}, standard deviation: {}, min: {}, max: {}'.format(
        feature_stats['pct_null'], feature_stats['mean'], feature_stats['sd'],
        feature_stats['min'], feature_stats['max']))

    
def inspect_cat_feat(feature):

    # Inspect percent missing and distribution of values for categorical feature
    feature_stats = {
        'pct_null': np.round(feature.isnull().sum() * 100 / len(feature), 2),
        'dist': feature.value_counts()}
    
    # Print feature summary statistics
    print('-'*40)
    print('Feature: {}'.format(feature.name))
    print('Percent null: {}'.format(feature_stats['pct_null']))
    print('Distribution of values:')
    print(feature_stats['dist'])


In [ ]:

# Inspect the continous features
continous_feat = ['Age', 'Annual_Premium', 'Vintage']
for feature in continous_feat:
    inspect_cont_feat(data[feature])

# Inspect the categorical features
categorical_feat = ['Gender', 'Driving_License', 'Region_Code', 'Previously_Insured', 
                    'Vehicle_Age', 'Vehicle_Damage', 'Policy_Sales_Channel']
for feature in categorical_feat:
    inspect_cat_feat(data[feature])


### Inspect outcome variable

In [ ]:
# Inspect outcome variable (missing values, distinct values, class imbalance)
inspect_cat_feat(data['Response'])

print('Interest variable, percent negative examples: {}, percent positive examples {}'.format(
    np.round((sum(data['Response'] == 0) / data.shape[0]) * 100, 2),
    np.round((sum(data['Response'] == 1) / data.shape[0]) * 100, 2)))

# Data cleaning and preprocessing

### Drop irrelevant features

In [ ]:
# Remove irrelevant features
del data['id']

### Cast binary string features to numeric

In [ ]:
# Cast Gender to numeric (Male => 0, Female => 1)
data['Gender'] = (data['Gender'] == 'Female').astype(int)

# Cast Vehicle_Damage to numeric (No => 0, Yes => 1)
data['Vehicle_Damage'] = (data['Vehicle_Damage'] == 'Yes').astype(int)

### Cast ordinal string features to numeric

In [ ]:
# Cast Vehicle_Age to numeric ('< 1 Year' => 0, '1-2 Year' => 1, '> 2 Years' => 2)
data.loc[data['Vehicle_Age'] == '< 1 Year', 'Vehicle_Age'] = '0'
data.loc[data['Vehicle_Age'] == '1-2 Year', 'Vehicle_Age'] = '1'
data.loc[data['Vehicle_Age'] == '> 2 Years', 'Vehicle_Age'] = '2'
data['Vehicle_Age'] = data['Vehicle_Age'].astype(int)

### One-hot encode categorical features

In [ ]:

def one_hot_encode_feat(data, categorical_feat):

    preprocessed_data = pd.DataFrame()

    for col_name in data.columns:
        if col_name in categorical_feat:
            # One-hot encode categorical features
            one_hot_feat = pd.get_dummies(data[col_name])
            one_hot_feat = pd.get_dummies(data[col_name], prefix=col_name)
            preprocessed_data = pd.concat((preprocessed_data, one_hot_feat), axis=1)
        else:
            preprocessed_data = pd.concat((preprocessed_data, data[col_name]), axis=1)

    return preprocessed_data


data = one_hot_encode_feat(data, ['Region_Code', 'Policy_Sales_Channel'])

In [ ]:
# Reorder columns
cols = ['Response'] \
    + [col for col in data.columns \
           if not (col.startswith('Region_Code_') or col.startswith('Policy_Sales_Channel_')) \
           and col != 'Response'] \
    + [col for col in data.columns if col.startswith('Region_Code_')] \
    + [col for col in data.columns if col.startswith('Policy_Sales_Channel_')]

data = data[cols]

In [ ]:
data.head()

### Split data into train, validation, and test sets

In [ ]:
# Shuffle data
data = data.sample(frac=1, random_state=1)

# Assign features, outcome to separate dataframes
X = data.copy()
del X['Response']
y = data['Response']

# Split data into training, validation, and test sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=1)
X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=0.5, random_state=1)

print(X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape)

### Inspect distribution of outcome across train, validation, and test sets

In [ ]:
# If classes in outcome are imbalanced, inspect class distribution of train, validation, and test set
label_counts = pd.DataFrame({
    'data_set': ['train', 'validation', 'test'],
    'percent_0': [np.round(np.mean(y), 5) for y in [y_train, y_val, y_test]],
    'percent_1': [np.round((1.0 - np.mean(y)), 5) for y in [y_train, y_val, y_test]],
    'count_0': [len(y[y == 0]) for y in [y_train, y_val, y_test]],
    'count_1': [len(y[y == 1]) for y in [y_train, y_val, y_test]]})
label_counts

### Perform min-max scaling for continous features

In [ ]:
continous_feat = ['Age', 'Annual_Premium', 'Vintage']

# Pull summary statistics from training set for min-max scaling and missing value imputation
train_col_max = X_train.max(axis=0)
train_col_min = X_train.min(axis=0)
train_col_mode = X_train.mode(axis=0).iloc[0, :]

# Scale continuous features using min and max from training examples
for data_split in [X_train, X_val, X_test]:
    for feat in continous_feat:
        data_split[feat] = (data_split[feat] - train_col_min[feat]) / (train_col_max[feat] - train_col_min[feat])

In [ ]:
X_train.head()

# Inspect preprocessed features

In [ ]:
print(X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape)

In [ ]:

for data_split in [('Train', X_train), ('Validation', X_val), ('Test', X_test)]:
    
    data_split_name, data_split = data_split

    print('-'*100)
    print('Dataset: {}'.format(data_split_name))
    
    # Inspect preprocessed continuous features
    for feature in ['Age', 'Annual_Premium', 'Vintage']:
        inspect_cont_feat(data_split[feature])

    # Inspect preprocessed categorical features
    for feature in ['Gender', 'Driving_License', 'Previously_Insured', 'Vehicle_Age', 'Vehicle_Damage']:
        inspect_cat_feat(data_split[feature])


### Identify highly correlated features

In [ ]:
# Inspect highly correlated features
for i in range(X_train.shape[1]):
    for j in range(i+1, X_train.shape[1]):
        col1, col2 = X_train.columns[i], X_train.columns[j]
        corr = np.round(X_train[col1].corr(X_train[col2]), 2)
        if np.abs(corr) > 0.3:
            print('Correlation between features {} and {} is {}'.format(col1, col2, corr))

# Train linear model

### Define evaluation metrics

In [ ]:

def eval_metrics(y_true, y_hat):
    # Return evaluation metrics
    
    # Generate confusion matrix
    conf_mat = pd.DataFrame(confusion_matrix(y_true, np.round(y_hat).astype(int), normalize='true'))
    conf_mat.columns = ['predicted_0', 'predicted_1']
    conf_mat.index = ['actual_0', 'actual_1']
    
    # Calculate the area under the curve of precision-recall plot
    precision, recall, thresholds = precision_recall_curve(y_true, y_hat)
    pr_auc = auc(recall, precision)

    return {
        'f1_score': f1_score(y_true, np.round(y_hat).astype(int)),
        'pr_auc': pr_auc,
        'confusion_matrix': conf_mat}


### Train logistic regression model

In [ ]:
# Train linear model
lin_model = sklearn.linear_model.LogisticRegression(class_weight='balanced')
lin_model.fit(X_train, y_train)

In [ ]:
# Inspect train, validation error metrics
y_hat_train = lin_model.predict_proba(X_train)[:, 1]
y_hat_val = lin_model.predict_proba(X_val)[:, 1]

train_eval_metrics = eval_metrics(y_train, y_hat_train)
val_eval_metrics = eval_metrics(y_val, y_hat_val)

print('-'*40)
print('Train evaluation metrics')
print('F1 score: {}'.format(train_eval_metrics['f1_score']))
print('AUC for Precision-Recall Curve: {}'.format(train_eval_metrics['pr_auc']))
print('Confusion matrix:')
print(train_eval_metrics['confusion_matrix'])

print('-'*40)
print('Validation evaluation metrics')
print('F1 score: {}'.format(val_eval_metrics['f1_score']))
print('AUC for Precision-Recall Curve: {}'.format(val_eval_metrics['pr_auc']))
print('Confusion matrix:')
print(val_eval_metrics['confusion_matrix'])

### Examine linear relationships between features and outcome

In [ ]:
# Manually add bias term for linear model by adding col with constant values
X_train_sm, X_val_sm = X_train.copy(), X_val.copy()
X_train_sm.loc[:, 'const'] = 1.0

# Train linear model for regression analysis
sm_lin_model = sm.Logit(y_train, X_train_sm).fit()

# Inspect coefficients and p-values for each features
sm_lin_model.summary()

# Train random forest model

In [ ]:

def train_rf_model(X, y, hyperparameters):

    rf_model = sklearn.ensemble.RandomForestClassifier(
        n_estimators=hyperparameters['n_estimators'], 
        max_features=hyperparameters['max_features'],
        max_depth=hyperparameters['max_depth'],
        class_weight=hyperparameters['class_weight'],
        bootstrap=hyperparameters['bootstrap'],
        # Parralelize training across all CPUs
        n_jobs=-1)

    rf_model.fit(X, y)
    
    return rf_model


### Hyperparameter tuning

In [ ]:

num_features = X_train.shape[1]

# Set default hyperparameter values
default_hp_vals = {
    'n_estimators': 100,
    'max_features': 'auto',
    'max_depth': 12,
    'class_weight': 'balanced',
    'bootstrap': True}

# Define hyperparameters to test
hp_test_vals = {
    'n_estimators': [5, 10, 25, 50, 100, 150, 200, 500],
    'max_features': ['auto', int(num_features * 0.5), int(num_features * 0.75), num_features],
    'max_depth': [None, 1, 3, 5, 8, 12, 25, 50],
    'class_weight': [None, 'balanced', 'balanced_subsample'],
    'bootstrap': [True, False]}

# Store experimental results in dataframe
rf_hp_tuning_results = pd.DataFrame(
    columns=['n_estimators', 'max_features', 'max_depth', 'class_weight', 'bootstrap', 
             'train_f1_score', 'train_pr_curve_auc', 'val_f1_score', 'val_pr_curve_auc'])

# Tune hyperparameters
for hp in hp_test_vals.keys():
    for hp_val in hp_test_vals[hp]:

        # Define experiment hyperparameter values
        exp_hp = default_hp_vals.copy()
        exp_hp[hp] = hp_val
        
        # Train RF model
        rf_model = train_rf_model(X_train, y_train, exp_hp)
        
        # Inspect train, validation error metrics
        y_hat_train = rf_model.predict_proba(X_train)[:, 1]
        y_hat_val = rf_model.predict_proba(X_val)[:, 1]
        train_eval_metrics = eval_metrics(y_train, y_hat_train)
        val_eval_metrics = eval_metrics(y_val, y_hat_val)
        
        # Record experiment metadata and eval metrics
        exp_results = {
            'n_estimators': exp_hp['n_estimators'],
            'max_features': exp_hp['max_features'],
            'max_depth': exp_hp['max_depth'],
            'class_weight': exp_hp['class_weight'],
            'bootstrap': exp_hp['bootstrap'],
            'train_f1_score': train_eval_metrics['f1_score'],
            'train_pr_curve_auc': train_eval_metrics['pr_auc'],
            'val_f1_score': val_eval_metrics['f1_score'],
            'val_pr_curve_auc': val_eval_metrics['pr_auc']}
        rf_hp_tuning_results = rf_hp_tuning_results.append(
            exp_results, ignore_index = True)


In [ ]:
rf_hp_tuning_results

### Train optimal random forest model

In [ ]:
# Define optimal hyperparameter values
rf_optimal_hp = {
    'n_estimators': 150,
    'max_features': int(num_features * 0.75),
    'max_depth': 12,
    'class_weight': 'balanced_subsample',
    'bootstrap': True}

# Train RF model
rf_model = train_rf_model(X_train, y_train, rf_optimal_hp)

# Inspect train, validation error metrics
y_hat_train = rf_model.predict_proba(X_train)[:, 1]
y_hat_val = rf_model.predict_proba(X_val)[:, 1]

train_eval_metrics = eval_metrics(y_train, y_hat_train)
val_eval_metrics = eval_metrics(y_val, y_hat_val)

print('-'*40)
print('Train evaluation metrics')
print('F1 score: {}'.format(train_eval_metrics['f1_score']))
print('AUC for Precision-Recall Curve: {}'.format(train_eval_metrics['pr_auc']))
print('Confusion matrix:')
print(train_eval_metrics['confusion_matrix'])

print('-'*40)
print('Validation evaluation metrics')
print('F1 score: {}'.format(val_eval_metrics['f1_score']))
print('AUC for Precision-Recall Curve: {}'.format(val_eval_metrics['pr_auc']))
print('Confusion matrix:')
print(val_eval_metrics['confusion_matrix'])


### Examine non-linear relationships between features and outcome

In [ ]:
# Inspect variable importance
rf_var_importance = pd.DataFrame({'feature': X_train.columns, 'importance': rf_model.feature_importances_})
rf_var_importance.sort_values(by='importance', ascending=False, inplace=True)
print(rf_var_importance.iloc[0:20, :])

# Train gradient boosting model

In [ ]:

# Convert Pandas dataframes to DMatrix class for XGBoost library
train_xgb = xgb.DMatrix(X_train, label=y_train)
val_xgb = xgb.DMatrix(X_val, label=y_val)
test_xgb = xgb.DMatrix(X_test, label=y_test)


def train_xgb_model(data, hyperparameters):
    
    # Define parameters for gradient boosting model
    param = {
        'objective':'binary:logistic',
        'scale_pos_weight': 5.1,  # Set to heuristic sum(neg examples) / sum(pos examples)
        'max_depth': hyperparameters['max_depth'], 
        'eta': hyperparameters['learning_rate'], 
        'max_delta_step': hyperparameters['max_delta_step'],
        'colsample_bytree': hyperparameters['num_features']}
    num_round = hyperparameters['n_round']

    # Train model
    xgb_model = xgb.train(param, data, num_round)

    return xgb_model


### Hyperparameter tuning

In [ ]:

# Set default hyperparameter values
default_hp_vals = {
    'learning_rate': 0.01,
    'n_round': 100,
    'max_depth': 12,
    'max_delta_step': 0,
    'num_features': 1}

# Define hyperparameter values to test
hp_test_vals = {
    'learning_rate': [0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1.0],
    'n_round': [5, 10, 25, 50, 100, 150, 200, 500],
    'max_depth': [1, 3, 5, 8, 12, 25], 
    'max_delta_step': [0, 1, 3, 5, 8, 10],
    'num_features': [1.0, 0.75, 0.5, 0.25]}

# Store experimental results in dataframe
xgb_hp_tuning_results = pd.DataFrame(
    columns=['learning_rate', 'n_round', 'max_depth', 'max_delta_step', 'num_features', 
             'train_f1_score', 'train_pr_curve_auc', 'val_f1_score', 'val_pr_curve_auc'])


# Tune hyperparameters
for hp in hp_test_vals.keys():
    for hp_val in hp_test_vals[hp]:

        # Define experiment hyperparameter values
        exp_hp = default_hp_vals.copy()
        exp_hp[hp] = hp_val
        
        # Train gradient boosting model
        xgb_model = train_xgb_model(train_xgb, exp_hp)
        
        # Inspect train, validation error metrics
        y_hat_train = xgb_model.predict(train_xgb)
        y_hat_val = xgb_model.predict(val_xgb)
        train_eval_metrics = eval_metrics(y_train, y_hat_train)
        val_eval_metrics = eval_metrics(y_val, y_hat_val)
        
        # Record experiment metadata and eval metrics
        exp_results = {
            'learning_rate': exp_hp['learning_rate'],
            'n_round': exp_hp['n_round'],
            'max_depth': exp_hp['max_depth'],
            'max_delta_step': exp_hp['max_delta_step'],
            'num_features': exp_hp['num_features'],
            'train_f1_score': train_eval_metrics['f1_score'],
            'train_pr_curve_auc': train_eval_metrics['pr_auc'],
            'val_f1_score': val_eval_metrics['f1_score'],
            'val_pr_curve_auc': val_eval_metrics['pr_auc']}
        xgb_hp_tuning_results = xgb_hp_tuning_results.append(
            exp_results,
            ignore_index = True)


In [ ]:
xgb_hp_tuning_results

### Train optimal gradient boosting model

In [ ]:

# Define optimal hyperparameter values
xgb_optimal_hp = {
    'learning_rate': .05,
    'n_round': 150,
    'max_depth': 12,
    'max_delta_step': 0,
    'num_features': 0.5}


# Train XGB model
xgb_model = train_xgb_model(train_xgb, xgb_optimal_hp)

# Inspect train, validation error metrics
y_hat_train = xgb_model.predict(train_xgb)
y_hat_val = xgb_model.predict(val_xgb)
train_eval_metrics = eval_metrics(y_train, y_hat_train)
val_eval_metrics = eval_metrics(y_val, y_hat_val)

print('-'*40)
print('Train evaluation metrics')
print('F1 score: {}'.format(train_eval_metrics['f1_score']))
print('AUC for Precision-Recall Curve: {}'.format(train_eval_metrics['pr_auc']))
print('Confusion matrix:')
print(train_eval_metrics['confusion_matrix'])

print('-'*40)
print('Validation evaluation metrics')
print('F1 score: {}'.format(val_eval_metrics['f1_score']))
print('AUC for Precision-Recall Curve: {}'.format(val_eval_metrics['pr_auc']))
print('Confusion matrix:')
print(val_eval_metrics['confusion_matrix'])


In [ ]:
# Inspect test set performance, generalizability
y_hat_test = xgb_model.predict(test_xgb)
test_eval_metrics = eval_metrics(y_test, y_hat_test)

print('-'*40)
print('Test evaluation metrics')
print('F1 score: {}'.format(test_eval_metrics['f1_score']))
print('AUC for Precision-Recall Curve: {}'.format(val_eval_metrics['pr_auc']))
print('Confusion matrix:')
print(test_eval_metrics['confusion_matrix'])

### Examine non-linear relationships between features and outcome

In [ ]:
# Examine feature importance for XGB model

# Get the impurity-based importance values for each feature
feature_imp = pd.DataFrame({'feature': [], 'importance': []})
for k, v in xgb_model.get_score(importance_type='gain').items():
    feature_imp = feature_imp.append({'feature': k, 'importance': v}, ignore_index = True)

# Normalize importance values
feature_imp['importance'] = (feature_imp['importance'] / np.sum(feature_imp['importance']))
feature_imp = feature_imp.sort_values(by='importance', ascending=False)

print(feature_imp.iloc[0:30, :])

# Export trained model

In [ ]:
# Save trained model, associated hyperparameter values, and test set eval metrics
model = {
    'model': xgb_model,
    'model_hyperparameters': xgb_optimal_hp,
    'test_set_evaluation_metrics': test_eval_metrics}

# Save to pickle file
pickle.dump(model, open("insurance_cross_sell_model.pkl", "wb"))